### 카카오 매장 블로그 후기 검색

In [2]:
# 필요한 모듈 참조
import requests
import json
from pandas import DataFrame
from pandas import read_excel 

In [3]:
# 접속할 URL
urlFmt = "https://dapi.kakao.com/v2/search/blog?query={query}#&page={page}&size={size}"


In [4]:
# 검색 지역 
location = '서울특별시 서초구 방배동 836-10 삼경빌딩'
loc = location.split(" ")
loc



['서울특별시', '서초구', '방배동', '836-10', '삼경빌딩']

In [5]:
# xlsx 파일 불러오기
store_list = None # 읽어들일 내용을 저장하기 위한 빈 변수 생성

df_y = read_excel("요기요 (서울특별시 서초구 방배동 836-10 삼경빌딩).xlsx",sheet_name='요기요')
df_y = df_y.sort_values(by='별점', ascending=False) 
df_100 = df_y.head(200) # 상위 100개 
df_100 = df_100.sort_values(by='리뷰', ascending=False) # 리뷰 순 오름차순 정렬
df_100_star = df_100.head(100) 
# df_100_star
df_100_star['매장명']
store_list1 = list(df_100_star['매장명'].values)
store_list1


['기대만족-본점',
 '벚꽃스시-상도점',
 '벚꽃스시-상도점',
 '반올림피자-사당방배점',
 '청년피자-사당방배점',
 '조이플',
 '모두뼈다귀감자탕-상도점',
 '롯데리아-남성역점',
 '우제홍참치-강남서초점',
 '샌드위치카페하이나',
 '무국적식탁-관악점',
 '무국적식탁-관악점',
 '동근이숯불두마리치킨-남성역점',
 '동근이숯불두마리치킨-남성역점',
 '동근이숯불두마리치킨-남성역점',
 '돈갓수제돈까스-사당점',
 '돈갓수제돈까스-사당점',
 '돈갓수제돈까스-사당점',
 '논현동황금식당',
 '청년곱도리탕-직영점',
 '지코바치킨-방배1호점',
 '따띠삼겹-사당점',
 '라사천마라탕-방배점',
 '라사천마라탕-방배점',
 '라사천마라탕-방배점',
 '구구족-이수역점',
 '국민낙곱새-이수점',
 '국민낙곱새-이수점',
 '엔제리너스-이수역점',
 '한식세끼1인김치찜&부대찌개-이수점',
 '한식세끼1인김치찜&부대찌개-이수점',
 '한식세끼1인김치찜&부대찌개-이수점',
 '한식세끼1인김치찜&부대찌개-이수점',
 '유캔두잇-서초교대점',
 '옐로우피자-사당방배점',
 '옐로우피자-사당방배점',
 '옐로우피자-사당방배점',
 '옐로우피자-사당방배점',
 '옐로우피자-사당방배점',
 '빙달-방배점',
 '백억커피-낙성대점',
 '냠냠족발&보쌈-서초점',
 '냠냠족발&보쌈-서초점',
 '제육대가-서초점',
 '제육대가-서초점',
 '국수나무-이수역점',
 '고급양식-교대점',
 '커피브라더-이수역점',
 '커피브라더-이수역점',
 '커피브라더-이수역점',
 '난바우동-방배본점',
 '던킨-교대법원점',
 '회가대표초밥24-남성역점',
 '백억커피-이수역점',
 '냉면&덮밥도배달돼지-방배사당점',
 '냉면&덮밥도배달돼지-방배사당점',
 '투썸플레이스-내방역점',
 '혜화동돈까스극장-서울방배점',
 '진수산',
 '스시로로',
 '스시로로',
 '육회바른연어-서초점',
 '육회바른연어-서초점',
 '육회바른연어-서초점',
 '육회바른연어-서초점',
 '과일프

In [54]:
# 검색 매장
store_list = []

for a in store_list1 :
    store_list.append(a[:a.find("-")])

store_list = list(set(store_list))
print(len(store_list),'개의 매장을 검색합니다',store_list)

# 최종 저장할 리스트 생성 
mylist2 = []

In [57]:
for i in range(len(store_list)) :
    # 요청변수
    query = f"{loc[1]} {loc[2]} {store_list[i]}"
    page = 1
    size = 50
    key = "87fa270de879f3dd014b8779623884db"

    # 접속 객체 생성
    session = requests.Session()

    # 접속객체에 부가정보(header) 삽입하기
    session.headers.update({
        "Authorization": "KakaoAK %s" % key    # 인증키
    })

    # 다음 페이지가 존재하는지 여부 --> False인 동안은 while문 반복
    isEnd = False

    # 반복 수행 도중 추출되는 검색결과 리스트를 결합할 빈 리스트 준비
    mylist = []

    # 1페이지부터 시작
    page = 1

    while not isEnd :
            # 접속할 URL
        url = urlFmt.format(query=query, page=page, size=size)
        url

        # 생성한 접속객체를 활용하여 API에 접속
        r = session.get(url)

        # 접속에 실패한 경우
        if r.status_code != 200:
            # 에러코드와 에러메시지 출력
            msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
            # 에러를 강제로 생성시킴
            raise Exception(msg)

        # 인코딩 형식 지정
        r.encoding = "utf-8"
        # 가져온 문자열을 딕셔너리로 변환
        data = json.loads(r.text)

        # 검색 결과를 갖고 있는 딕셔너리를 포함하는 리스트 추출
        searchList = data['documents']

        for j in range(len(searchList)) :
            searchList[j]['storeName'] = store_list[i]
            searchList[j]['contents'] = (searchList[j]['contents'].replace('</b>','')).replace('<b>','')
            searchList[j]['title'] = (searchList[j]['title'].replace('</b>','')).replace('<b>','')

        # 추출한 리스트를 미리 준비한 빈 리스트에 추가
        mylist.extend(searchList)

        # 증감식에 해당하는 isEnd변수 추출
        isEnd = bool(data['meta']['is_end'])
        page += 1
        print("isEnd=", isEnd, ", 다음페이지=", page)
        # if page > 200 : # 200개를 가져오는건 뭐야 ;;;;; 
        #     break
    
    mylist2.extend(mylist)
mylist2 = list(set(mylist2))
    

Exception: [429 Error] Too Many Requests 에러가 발생함

In [56]:
# 결과 엑셀 저장 및 출력
df = DataFrame(mylist2) 
#df.drop(['thumbnail'],axis=1,inplace=True)
df.to_excel("카카오 블로그 검색 결과(%s).xlsx" % loc[2],sheet_name='검색',index=False)
df

,blogname,contents,datetime,thumbnail,title,url,storeName
0,Dream & Do it,"옛날에 산책하러 종종 왔던 곳인데, 봄에는 처음와본다 ㅎㅎ 4월에는 벚꽃축제를 매년...",2023-04-04T11:23:00.000+09:00,https://search4.kakaocdn.net/argon/130x130_85_...,"벚꽃놀이🌸, 사당역 데이트 코스 추천 #교꾸스시 #태양커피",https://blog.naver.com/daily-project/223064259840,벚꽃스시
1,"삶, 자연, 예술",난 벚꽃 명소보다 이런 소소한 곳이 더 좋더라​​ ​ ​ 3월은 워낙 바빠서 문화생...,2023-05-22T21:11:00.000+09:00,https://search3.kakaocdn.net/argon/130x130_85_...,[일상] 2-3월. 바빴지만 행복했다,https://blog.naver.com/koszerg4/223108953381,벚꽃스시
2,벼농사 수확기,왜인지 벚꽃이 엄청 빨리 폈다! ​ 올해 생일의 원피스는 Plan de Paris ...,2023-04-08T13:41:00.000+09:00,https://search1.kakaocdn.net/argon/130x130_85_...,"기뻤던 일 정리!(쵸이닷 런치, 더라운드 청담 북경오리, 정식카페, 스시코우지 3월...",https://blog.naver.com/commutinglawschooler/22...,벚꽃스시
3,호두나무가 사는 세상,주말 아침때였는데요 요즘 너무 추워서 그런지 어디 가가 귀찮더라고요 방배동 카페골목...,2022-12-21T13:45:00.000+09:00,https://search3.kakaocdn.net/argon/130x130_85_...,방배동 카페골목 원초적인 이름을 가진 초밥집,https://blog.naver.com/hyuni9987/222961735506,벚꽃스시
4,LABDANUM18,빵집 서초구 방배동에도 지점이 있었다는거 알고계심니카… 지금도 있는진 모르겠네 ​ ...,2023-03-30T22:16:00.000+09:00,https://search2.kakaocdn.net/argon/130x130_85_...,/ 3월 3주의 일상 - 도쿄여행(2),https://blog.naver.com/labdanum18/223060302064,벚꽃스시
...,...,...,...,...,...,...,...
1998,방배댁의 데일리,살랑대고 벚꽃이 만발하니 자꾸 외출하고 싶어지고 외식하고 싶어시고 ... ​ 오늘은...,2022-04-07T16:27:00.000+09:00,https://search3.kakaocdn.net/argon/130x130_85_...,(방배동 맛집) 방배먹자골목 김태완스시,https://blog.naver.com/braque83/222694340941,벚꽃스시
1999,오백이 PLAYLIST,22년 3월 방문 주소: 서울 서초구 방배중앙로 188 방배동 760-2 25년 이...,2022-03-26T18:44:00.000+09:00,https://search2.kakaocdn.net/argon/130x130_85_...,"방배동 카페 골목, 푸짐한 초밥집",https://blog.naver.com/bnewmind/222683636446,벚꽃스시
2000,솜구의 하루,패티가 진짜 예술 근데 양이 많아서 사이드는 굳이 시킬 필요가 없다 ​ ​ 사무실에...,2021-09-16T20:22:00.000+09:00,https://search2.kakaocdn.net/argon/130x130_85_...,방배동 출근러의 맛집 추천 -1,https://blog.naver.com/som2570/222508079100,버거파이터
2001,쓸모있는 집돌이,특별시 서초구 방배동[4] 국적 대한민국 국기 대한민국 본관 수원 백씨 (水原 白氏...,2023-02-28T07:26:00.000+09:00,https://search4.kakaocdn.net/argon/130x130_85_...,백종원 예산시장 창업자 위생 식자재 관리 맛집 통닭 주차장 막걸리 바베큐 칼국수 휴장,https://blog.naver.com/braveship/223030023537,버거파이터
